In [1]:
from non_dominated_sort import fast_non_dominated_sort as fnds
from non_dominated_sort import dominance_check_between_two_points as dc
from non_dominated_sort import insert as insert
from non_dominated_sort import non_dominated_sorting as nds
from fitness_calculation import calculate_error_rate as error
from one_bit_purifying_search import one_bit_purifying_search as obps

In [2]:
import pandas as pd
import io
from sklearn.model_selection import train_test_split
import random
from sklearn import preprocessing

import math
from operator import itemgetter

In [3]:
from pymoo.indicators.hv import HV
import statistics

In [4]:
pd.set_option("display.max_columns",100)

In [5]:
colnames = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15']
df = pd.read_csv('australian.txt', delimiter=r"\s+",  names=colnames, encoding='utf-8', index_col=False, header=None)
#df_test = pd.read_csv('test.csv', sep=',', encoding='utf-8', index_col=False)
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
0,1,22.08,11.46,2,4,4,1.585,0,0,0,1,2,100,1213,0
1,0,22.67,7.00,2,8,4,0.165,0,0,0,0,2,160,1,0
2,0,29.58,1.75,1,4,4,1.250,0,0,0,1,2,280,1,0
3,0,21.67,11.50,1,5,3,0.000,1,1,11,1,2,0,1,1
4,1,20.17,8.17,2,6,4,1.960,1,1,14,0,2,60,159,1


In [6]:
# df = df.drop('Unnamed: 0', axis=1)
# df_test = df_test.drop('Unnamed: 0', axis=1)
# df.head(2)

In [7]:
df.shape

(690, 15)

In [8]:
df['target'] = df['A1']
df = df.drop(['A1'], axis=1)

In [9]:
# df_test['target'] = df_test['Creditability']
# df_test = df_test.drop(['Creditability'], axis=1)

In [10]:
print(df.shape)
#print(df_test.shape)

(690, 15)


In [11]:
df['target'].value_counts()

target
1    468
0    222
Name: count, dtype: int64

In [12]:
# df_test['target'].value_counts()

In [13]:
df.head(1)

,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,target
0,22.08,11.46,2,4,4,1.585,0,0,0,1,2,100,1213,0,1


# Creating dummy variables

In [14]:
# train_up = pd.get_dummies(df, columns = ['Account.Balance', 'Payment.Status.of.Previous.Credit', 'Purpose', 'Sex...Marital.Status', 'Guarantors', 'Most.valuable.available.asset', 'Concurrent.Credits', 'Type.of.apartment', 'Occupation', 'Foreign.Worker', 'Telephone'], drop_first= True)
# test_up = pd.get_dummies(df_test, columns = ['Account.Balance', 'Payment.Status.of.Previous.Credit', 'Purpose', 'Sex...Marital.Status', 'Guarantors', 'Most.valuable.available.asset', 'Concurrent.Credits', 'Type.of.apartment', 'Occupation', 'Foreign.Worker', 'Telephone'], drop_first= True)


In [15]:
# print(train_up.shape)
# print(test_up.shape)

In [16]:
# train_up_null_count = train_up.isna().sum()
# train_up_null_count = train_up_null_count[train_up_null_count>0]
# train_up_null_count

In [17]:
# train_up = train_up.dropna()

In [18]:
# train_up.shape

In [19]:
# train_up.head(1)

In [20]:
# test_up_null_count = test_up.isna().sum()
# test_up_null_count = test_up_null_count[test_up_null_count>0]
# test_up_null_count

In [21]:
# test_up = test_up.dropna()

In [22]:
# test_up.shape

In [23]:
# test_up.head(1)

# Creating training test split

In [24]:
X = df.drop('target', axis=1)
y = df['target']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,y , random_state=42,  test_size=0.30,  shuffle=True) 

In [26]:
# X_train = train_up.drop('target', axis=1)
# y_train = df['target']

# X_test = test_up.drop('target', axis=1)
# y_test = df_test['target']

In [27]:
X_test.columns

Index(['A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12',
       'A13', 'A14', 'A15'],
      dtype='object')

In [28]:
X_train.shape

(483, 14)

In [29]:
X_test.shape

(207, 14)

In [30]:
y_train.value_counts()

target
1    325
0    158
Name: count, dtype: int64

In [31]:
y_test.value_counts()

target
1    143
0     64
Name: count, dtype: int64

In [32]:
def select_population_for_next_gen(population_size, P_t_plus_1_vector_fitness): 
    print("population_size" + str(population_size))
    
    for i in range(len(P_t_plus_1_vector_fitness)):
        print("list(map(itemgetter(0), P_t_plus_1_vector_fitness)) list(map(itemgetter(0), P_t_plus_1_vector_fitness))")
        print(list(map(itemgetter(0), P_t_plus_1_vector_fitness))[i])
        if sum(list(map(itemgetter(0), P_t_plus_1_vector_fitness))[i])==0:
            P_t_plus_1_vector_fitness.pop(i)
    
    print("len of P_t_plus_1_vector_fitness" + str(len(P_t_plus_1_vector_fitness)))
    P_t_plus_1 =  list(map(itemgetter(0), P_t_plus_1_vector_fitness))
    P_t_plus_1_fitness = list(map(itemgetter (1,2), P_t_plus_1_vector_fitness))
    P_t_plus_1_fitness = [list (item) for item in P_t_plus_1_fitness]
    
    sorted_index, sorted_values = fnds(data = P_t_plus_1_fitness)
    print("sorted_index")
    print(sorted_index)
    index_to_go_for_next_gen  = []
    
    for i in range(len(sorted_index)-1):
        if (len(sorted_index[i]) <= population_size - len(index_to_go_for_next_gen)):
            print("first if")
            print(len(sorted_index[i]))
            print(population_size - len(index_to_go_for_next_gen))
            index_to_go_for_next_gen = index_to_go_for_next_gen + sorted_index[i] 
        elif(len(index_to_go_for_next_gen) < population_size):
            places_left = population_size - len(index_to_go_for_next_gen)
            print("places_left")
            print(places_left)
            index_to_go_for_next_gen = index_to_go_for_next_gen + sorted_index[i][0:places_left] 
        else:
            print("neithr if")
    print("vedved index_to_go_for_next_gen")
    print(index_to_go_for_next_gen)
    population_for_next_gen = [P_t_plus_1_vector_fitness[i] for i in index_to_go_for_next_gen] 
    print("loop population_for_next_gen")
    print(population_for_next_gen)
    return population_for_next_gen

In [33]:
def top_front(P_t_plus_1_vector_fitness): 
    print("P_t_plus_1_vector_fitness")
    print(P_t_plus_1_vector_fitness)
    P_t_plus_1 = list(map(itemgetter(0), P_t_plus_1_vector_fitness))
    P_t_plus_1_fitness = list(map(itemgetter(1,2), P_t_plus_1_vector_fitness))
    P_t_plus_1_fitness = [list(item) for item in P_t_plus_1_fitness]
    
    print("P_t_plus_1_fitness")
    print(P_t_plus_1_fitness)
    print("before fnds")
    sorted_index, sorted_values = fnds(data = P_t_plus_1_fitness)
    print("after fnds")
    sorted_first_front_by_error = sorted(sorted_values[0], key = lambda x: x[0], reverse=True) 
    index_of_minimum_error_element = sorted_index[0][sorted_values[0].index(sorted_first_front_by_error[0])] 
     
    top_front = [P_t_plus_1[i] for i in sorted_index[0]]
    optimal_sol =  P_t_plus_1[index_of_minimum_error_element]
       
    return top_front

In [34]:
def optimal_sol(P_t_plus_1_vector_fitness): 
    P_t_plus_1 = list(map(itemgetter(0), P_t_plus_1_vector_fitness))
    P_t_plus_1_fitness = list(map(itemgetter(1,2), P_t_plus_1_vector_fitness))
    P_t_plus_1_fitness = [list(item) for item in P_t_plus_1_fitness]
    
    sorted_index, sorted_values = fnds(data = P_t_plus_1_fitness)
    sorted_first_front_by_error = sorted(sorted_values[0], key = lambda x: x[0], reverse=True) 
    index_of_minimum_error_element = sorted_index[0][sorted_values[0].index(sorted_first_front_by_error[0])] 
    
    top_front = [P_t_plus_1[i] for i in sorted_index[0]]
    optimal_sol =  P_t_plus_1[index_of_minimum_error_element]
    optimal_sol_fitness =  P_t_plus_1_fitness[index_of_minimum_error_element]
    
    return optimal_sol, optimal_sol_fitness

In [35]:
def MOFS(gen_max, obps_freq, dataset_train, target_train, dataset_test, target_test, initial_population):
    P_0 = initial_population
    all_gen_population_and_fitness = []
    num_feature = 14
    
    HV_list = []
    ref_point = np.array([14,1])
    mean_hv_of_prev_10_gen_list=[]
    difference_of_current_hv_and_mean_last_10_gen_list = []
    converged = False
    
    for t in (range(gen_max)):
        print("Attention - Generation " +str(t+1) + " starts")
        P_t_plus_1 = []
        P_t_plus_1_vector_fitness = []
        P_t_plus_1_vector = []
        P_t_plus_1_fitness = []
        round_robin_flag = 0
        N = len(P_0)
        print("NNNNNNNNNNNNNNNNNNNNNN")
        print(N)
        
        for i in range(1,N+1):
            random.seed(t+30*i)
            r1, r2, r3 = random.sample(range(0,N), 3)
            print(r1, r2, r3)
            print(P_0[r1],P_0[r2],P_0[r3])
            X_r1, num_feature_r1, f1_score_r1 = error(dataset_train=X_train, target_train = y_train, dataset_test = X_test, target_test = y_test, X_rn = P_0[r1])
            X_r2, num_feature_r2, f1_score_r2 = error(dataset_train=X_train, target_train = y_train, dataset_test = X_test, target_test = y_test, X_rn = P_0[r2])
            X_r3, num_feature_r3, f1_score_r3 = error(dataset_train=X_train, target_train = y_train, dataset_test = X_test, target_test = y_test, X_rn = P_0[r3])
            
            Dict_vector_cardinality_accuracy = {(num_feature_r1, f1_score_r1):X_r1, (num_feature_r2, f1_score_r2):X_r2, (num_feature_r3, f1_score_r3):X_r3 }
            num_feature_and_accuracy_for_3_candidate_sol = [[num_feature_r1, f1_score_r1], [num_feature_r2, f1_score_r2], [num_feature_r3, f1_score_r3]]
            
            res, smallest_key = nds(num_feature_and_accuracy_for_3_candidate_sol)
            res_smallest_key = res[smallest_key]
            
            res_smallest_key = sorted(res_smallest_key, key=lambda x:x[1], reverse = False)
            round_robin_flag = round_robin_flag + 1
            
            if(res_smallest_key[0] + res_smallest_key[0] != 0):
                X_best_t = res_smallest_key[0]
            else:
                X_best_t = res_smallest_key[1]
            
            removed_ = Dict_vector_cardinality_accuracy[tuple(X_best_t)]
            
            Dict_vector_cardinality_accuracy_after_removing_best = Dict_vector_cardinality_accuracy.copy()
            Dict_vector_cardinality_accuracy_after_removing_best.pop(tuple(X_best_t))
            
            list_of_Dict_vector_cardinality_accuracy_after_removing_best = list(Dict_vector_cardinality_accuracy_after_removing_best.items())
            
            if(len(list_of_Dict_vector_cardinality_accuracy_after_removing_best) == 0):
                X_r1_new = Dict_vector_cardinality_accuracy[tuple(X_best_t)]
                X_r2_new = Dict_vector_cardinality_accuracy[tuple(X_best_t)]
            elif(len(list_of_Dict_vector_cardinality_accuracy_after_removing_best) == 1):
                X_r1_new = list_of_Dict_vector_cardinality_accuracy_after_removing_best[0][1]
                X_r2_new = list_of_Dict_vector_cardinality_accuracy_after_removing_best[0][1]
            else:
                X_r1_new = list_of_Dict_vector_cardinality_accuracy_after_removing_best[0][1]
                X_r2_new = list_of_Dict_vector_cardinality_accuracy_after_removing_best[1][1]
                
            base_vector = Dict_vector_cardinality_accuracy[tuple(X_best_t)]
            X_i = P_0[i-1]
            
            X_ri, num_feature_ri, f1_score_ri = error(dataset_train=X_train, target_train = y_train, dataset_test = X_test, target_test = y_test, X_rn = X_i)
            X_rbase, num_feature_rbase, f1_score_rbase = error(dataset_train=X_train, target_train = y_train, dataset_test = X_test, target_test = y_test, X_rn = base_vector)
            
            xi_dominates_base, base_dominates_xi, base_and_xi_are_nondominating = dc(point1=[num_feature_rbase, f1_score_rbase], point2 =[num_feature_ri, f1_score_ri])
            
            X_r1_for_xor = X_r1_new.copy()
            X_r2_for_xor = X_r2_new.copy()
            
            sigma = 0.5
            c_i = [0]*num_feature
            v_i = [0]*num_feature
            F=0.1
            random.seed(t+30*i)
            random_val = random.sample(range(0,100), 1)
            random_value = random_val[0]/100
            
            if(base_dominates_xi == True):
                c_i = [sigma]*num_feature
            else:
                for i in range(num_feature):
                    c_i[i] = min(1, (F*int((bool(X_r1_for_xor[i]) ^ bool(X_r2_for_xor[i])))+sigma))
            mu = math.exp(-math.sqrt(t)) #t is generation count
            for i in range(num_feature):
                v_i[i] = mu * (math.floor((X_r1[i] + X_r2[i] + X_r3[i])/1.5)) + (1-mu) *((base_vector[i] * int(bool(X_r1_new[i]) ^ bool(X_r2_new[i]))) + (int(not(bool(base_vector[i]))) * int(bool(X_r1_new[i]) & bool(X_r2_new[i]))))
                if(v_i[i] < 0.5):
                    v_i[i] = 0
                else:
                    v_i[i] = 1
                
            if sum(v_i)>0:
                pass
            else:
                v_i = base_vector
                
            random.seed(t+30*i)

            u_0_1 = random.sample(range(0,100), 1)[0]
            u_0_1 = u_0_1/100

            random.seed(t+30*i)
            h=random.sample(range(0,N),1)[0]
            cr=0.5
            u_i = [0]*num_feature
                
            for i in range(num_feature):
                if(u_0_1 < cr or i==h):
                    u_i[i] = v_i[i]
                else:
                    u_i[i] = X_i[i]
                        
            X_ui, num_feature_ui, f1_score_ui = error(dataset_train=X_train, target_train = y_train, dataset_test = X_test, target_test = y_test, X_rn = u_i)
            xi_dominates_ui, ui_dominates_xi, ui_and_xi_are_nondominating = dc(point1=[num_feature_ui, f1_score_ui], point2 =[num_feature_ri, f1_score_ri])

            if(xi_dominates_ui == 1):
                print("xi_dominates_ui")
                P_t_plus_1_vector_fitness.append([X_ri, num_feature_ri, f1_score_ri])
            elif(ui_dominates_xi == 1):
                print("ui_dominates_xi")
                P_t_plus_1_vector_fitness.append([X_ui, num_feature_ui, f1_score_ui])
            else:
                print("xi ui nondominating")
                P_t_plus_1_vector_fitness.append([X_ri, num_feature_ri, f1_score_ri])
                P_t_plus_1_vector_fitness.append([X_ui, num_feature_ui, f1_score_ui])
        
        print("P_t_plus_1_vector_fitnessP_t_plus_1_vector_fitnessP_t_plus_1_vector_fitnessP_t_plus_1_vector_fitness")
        print(P_t_plus_1_vector_fitness)
        
        #Remove duplicates
        P_t_plus_1_vector_fitness_unique=[]
        for i in P_t_plus_1_vector_fitness:
            if i not in P_t_plus_1_vector_fitness_unique:
                P_t_plus_1_vector_fitness_unique.append(i)
                
        dup_count = len(P_t_plus_1_vector_fitness) - len(P_t_plus_1_vector_fitness_unique)
        #print(dup_count)
        
        print("P_t_plus_1_vector_fitness")
        print(P_t_plus_1_vector_fitness)
        print("P_t_plus_1_vector_fitness_unique")
        print(P_t_plus_1_vector_fitness_unique)
        
        P_t_plus_1_vector_fitness = P_t_plus_1_vector_fitness_unique.copy()
        
        if(gen_max/(obps_freq+1) == math.ceil(gen_max/(obps_freq+1))):
            top_front_ = top_front(P_t_plus_1_vector_fitness=P_t_plus_1_vector_fitness)
            print("top_front_")
            print(top_front_)
            optimal_sol_, optimal_sol_fitness_ = optimal_sol(P_t_plus_1_vector_fitness=P_t_plus_1_vector_fitness)
            obps_result = obps(X_train, X_test, y_train, y_test, S=top_front_, optimal_sol = optimal_sol_)
                        
            to_be_removed_sol = optimal_sol_.copy()
            to_be_removed_fit = optimal_sol_fitness_.copy()
                       
            to_be_removed_sol = [to_be_removed_sol, to_be_removed_fit[0], to_be_removed_fit[1]]
            
            P_t_plus_1_vector_fitness.remove(to_be_removed_sol)
            P_t_plus_1_vector_fitness.append(obps_result[0])
        print("vedP_t_plus_1_vector_fitness")
        print(P_t_plus_1_vector_fitness)
        print("len of P_t_plus_1_vector_fitness")
        print(len(P_t_plus_1_vector_fitness))
        
        non_0_len = len(P_t_plus_1_vector_fitness)
        for i in range(len(P_t_plus_1_vector_fitness)):
            if(i<non_0_len):
                if sum(list(map(itemgetter(0), P_t_plus_1_vector_fitness))[i])==0:
                    P_t_plus_1_vector_fitness_non_0 = P_t_plus_1_vector_fitness[:i] + P_t_plus_1_vector_fitness[i+1:]
                    non_0_len = non_0_len-1
                    i=i-1
                    P_t_plus_1_vector_fitness = P_t_plus_1_vector_fitness_non_0.copy()    

        print(N)
        if len(P_t_plus_1_vector_fitness) > N:
            population_for_next_gen_ = select_population_for_next_gen(population_size=N, P_t_plus_1_vector_fitness=P_t_plus_1_vector_fitness)
        else:
            population_for_next_gen_ = P_t_plus_1_vector_fitness
            
        
        print("population_for_next_gen_population_for_next_gen_")
        print(population_for_next_gen_)
        all_gen_population_and_fitness.append(population_for_next_gen_)
        
        P_0 = list(map(itemgetter(0), population_for_next_gen_))
        
        if(len(P_0)<3):
            print("break due to population size of less than 3")
            break
        
        print("Checking for convergence starts")
        P_0_metric = list(map(itemgetter(1,2), population_for_next_gen_))
        ind = HV(ref_point=ref_point)
        #print("HV for gen" + str(i+1), ind(np.array(P_0_metric)))
        HV_list.append(ind(np.array(P_0_metric)))
        print("HV_list till now is ")
        print(HV_list)
        
        
        #for i in range(len(HV_list)):
        if len(HV_list) > 10:
            i = len(HV_list)-1
            print(f"HV list has {i+1} elements now")
            mean_hv_of_prev_10_gen = statistics.mean(HV_list[i-10:i])
            print(f"mean_hv_of_prev_10_gen {mean_hv_of_prev_10_gen}")
            difference_of_current_hv_and_mean_last_10_gen = HV_list[i] - mean_hv_of_prev_10_gen
            print(f"difference_of_current_hv_and_mean_last_10_gen {difference_of_current_hv_and_mean_last_10_gen}")

            mean_hv_of_prev_10_gen_list.append(mean_hv_of_prev_10_gen)
            difference_of_current_hv_and_mean_last_10_gen_list.append(difference_of_current_hv_and_mean_last_10_gen)

            print("mean_hv_of_prev_10_gen_list")
            print(mean_hv_of_prev_10_gen_list)
            print("difference_of_current_hv_and_mean_last_10_gen_list")
            print(difference_of_current_hv_and_mean_last_10_gen_list)
            
            if(len(difference_of_current_hv_and_mean_last_10_gen_list)>=10):
                j=len(difference_of_current_hv_and_mean_last_10_gen_list)
                if(sum(difference_of_current_hv_and_mean_last_10_gen_list[j-10:j])<=0):
                    print(f"at gen {i+1} sum(difference_of_current_hv_and_mean_last_10_gen_list[j-10:j]) is less than zero, i.e., {sum(difference_of_current_hv_and_mean_last_10_gen_list[j-10:j])}")
                    print("difference_of_current_hv_and_mean_last_10_gen_list[j-10:j]")
                    print(difference_of_current_hv_and_mean_last_10_gen_list[j-10:j])
                    converged = True
                    print("converged is true now")
                    break
                else:
                    print(f"at gen {i+1} sum(difference_of_current_hv_and_mean_last_10_gen_list[j-10:j]) is NOT less than zero, i.e., {sum(difference_of_current_hv_and_mean_last_10_gen_list[j-10:j])}")
                    print("difference_of_current_hv_and_mean_last_10_gen_list[j-10:j]")
                    print(difference_of_current_hv_and_mean_last_10_gen_list[j-10:j])
                    print(f"at gen {i+1} is is not converged")
                    continue
            print("Checking for convergence ends")
                    
        if (converged == True):
            break
        
    return P_t_plus_1_vector_fitness, all_gen_population_and_fitness

In [36]:
import numpy as np
import random
Feature_population = []
rng = np.random.default_rng(14) #number of features
for i in range(50): #number of candidate solution in initial population
    array_of_0_1 = rng.random(14) #number of features
    array_of_0_1[array_of_0_1>=0.5] = 1
    array_of_0_1[array_of_0_1<0.5] = 0
    array_of_0_1=array_of_0_1.astype(int)
    list_of_0_1 = list(array_of_0_1)
    
    Feature_population.append(list_of_0_1)

In [37]:
len(Feature_population)

50

In [38]:
print(Feature_population)

[[1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1], [0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1], [0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0], [0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1], [1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0], [0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0], [0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1], [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1], [0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0], [1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1], [1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0], [0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0], [0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1], [1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0], [1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1], [1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1], [1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1], [0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1], [1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1], [1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 

In [39]:
last_gen_vector_fitness, all_generation_population_and_fitness = MOFS(gen_max=100, obps_freq=101, dataset_train=X_train, target_train = y_train, dataset_test = X_test, target_test = y_test, initial_population = Feature_population)

Attention - Generation 1 starts
NNNNNNNNNNNNNNNNNNNNNN
50
34 18 39
[0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0] [1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1] [0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0]
f1 score is-0.7344262295081967
accuracy is0.6086956521739131
error rate is0.3913043478260869
roc suc score is0.5009833916083917
gini is0.001966783216783341
[[ 14  50]
 [ 31 112]]
              precision    recall  f1-score   support

           0       0.31      0.22      0.26        64
           1       0.69      0.78      0.73       143

    accuracy                           0.61       207
   macro avg       0.50      0.50      0.50       207
weighted avg       0.57      0.61      0.59       207

f1 score is-0.7483443708609272
accuracy is0.6328502415458938
error rate is0.36714975845410625
roc suc score is0.5357298951048951
gini is0.07145979020979021
[[ 18  46]
 [ 30 113]]
              precision    recall  f1-score   support

           0       0.38      0.28      0.32        64
          

f1 score is-0.7162162162162162
accuracy is0.5942028985507246
error rate is0.4057971014492754
roc suc score is0.5034418706293706
gini is0.006883741258741249
[[ 17  47]
 [ 37 106]]
              precision    recall  f1-score   support

           0       0.31      0.27      0.29        64
           1       0.69      0.74      0.72       143

    accuracy                           0.59       207
   macro avg       0.50      0.50      0.50       207
weighted avg       0.58      0.59      0.58       207

f1 score is-0.7434210526315789
accuracy is0.6231884057971014
error rate is0.37681159420289856
roc suc score is0.5201048951048951
gini is0.04020979020979021
[[ 16  48]
 [ 30 113]]
              precision    recall  f1-score   support

           0       0.35      0.25      0.29        64
           1       0.70      0.79      0.74       143

    accuracy                           0.62       207
   macro avg       0.52      0.52      0.52       207
weighted avg       0.59      0.62      0.60

f1 score is-0.7407407407407408
accuracy is0.6280193236714976
error rate is0.3719806763285024
roc suc score is0.5408653846153846
gini is0.08173076923076916
[[ 20  44]
 [ 33 110]]
              precision    recall  f1-score   support

           0       0.38      0.31      0.34        64
           1       0.71      0.77      0.74       143

    accuracy                           0.63       207
   macro avg       0.55      0.54      0.54       207
weighted avg       0.61      0.63      0.62       207

f1 score is-0.7560137457044672
accuracy is0.6570048309178744
error rate is0.3429951690821256
roc suc score is0.5877403846153846
gini is0.17548076923076916
[[ 26  38]
 [ 33 110]]
              precision    recall  f1-score   support

           0       0.44      0.41      0.42        64
           1       0.74      0.77      0.76       143

    accuracy                           0.66       207
   macro avg       0.59      0.59      0.59       207
weighted avg       0.65      0.66      0.65  

f1 score is-0.740484429065744
accuracy is0.6376811594202898
error rate is0.3623188405797102
roc suc score is0.5694383741258742
gini is0.13887674825174834
[[ 25  39]
 [ 36 107]]
              precision    recall  f1-score   support

           0       0.41      0.39      0.40        64
           1       0.73      0.75      0.74       143

    accuracy                           0.64       207
   macro avg       0.57      0.57      0.57       207
weighted avg       0.63      0.64      0.64       207

f1 score is-0.726643598615917
accuracy is0.6183574879227053
error rate is0.3816425120772947
roc suc score is0.5468203671328671
gini is0.09364073426573416
[[ 23  41]
 [ 38 105]]
              precision    recall  f1-score   support

           0       0.38      0.36      0.37        64
           1       0.72      0.73      0.73       143

    accuracy                           0.62       207
   macro avg       0.55      0.55      0.55       207
weighted avg       0.61      0.62      0.62    

f1 score is-0.7533333333333333
accuracy is0.642512077294686
error rate is0.35748792270531404
roc suc score is0.5513548951048951
gini is0.10270979020979021
[[ 20  44]
 [ 30 113]]
              precision    recall  f1-score   support

           0       0.40      0.31      0.35        64
           1       0.72      0.79      0.75       143

    accuracy                           0.64       207
   macro avg       0.56      0.55      0.55       207
weighted avg       0.62      0.64      0.63       207

f1 score is-0.7235494880546076
accuracy is0.6086956521739131
error rate is0.3913043478260869
roc suc score is0.5268793706293706
gini is0.05375874125874125
[[ 20  44]
 [ 37 106]]
              precision    recall  f1-score   support

           0       0.35      0.31      0.33        64
           1       0.71      0.74      0.72       143

    accuracy                           0.61       207
   macro avg       0.53      0.53      0.53       207
weighted avg       0.60      0.61      0.60  

...

C:\Users\Ved\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ved\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ved\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ved\anaconda3\Lib\site-packages\sklearn\metrics\_cla

f1 score is-0.794701986754967
accuracy is0.7004830917874396
error rate is0.2995169082125604
roc suc score is0.6148929195804196
gini is0.22978583916083917
[[ 25  39]
 [ 23 120]]
              precision    recall  f1-score   support

           0       0.52      0.39      0.45        64
           1       0.75      0.84      0.79       143

    accuracy                           0.70       207
   macro avg       0.64      0.61      0.62       207
weighted avg       0.68      0.70      0.69       207

f1 score is-0.8171428571428571
accuracy is0.6908212560386473
error rate is0.3091787439613527
roc suc score is0.5
gini is0.0
[[  0  64]
 [  0 143]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        64
           1       0.69      1.00      0.82       143

    accuracy                           0.69       207
   macro avg       0.35      0.50      0.41       207
weighted avg       0.48      0.69      0.56       207

xi ui nondominating
5 

C:\Users\Ved\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ved\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ved\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ved\anaconda3\Lib\site-packages\sklearn\metrics\_cla

f1 score is-0.79734219269103
accuracy is0.7053140096618358
error rate is0.2946859903381642
roc suc score is0.6227054195804196
gini is0.24541083916083917
[[ 26  38]
 [ 23 120]]
              precision    recall  f1-score   support

           0       0.53      0.41      0.46        64
           1       0.76      0.84      0.80       143

    accuracy                           0.71       207
   macro avg       0.65      0.62      0.63       207
weighted avg       0.69      0.71      0.69       207

f1 score is-0.8171428571428571
accuracy is0.6908212560386473
error rate is0.3091787439613527
roc suc score is0.5
gini is0.0
[[  0  64]
 [  0 143]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        64
           1       0.69      1.00      0.82       143

    accuracy                           0.69       207
   macro avg       0.35      0.50      0.41       207
weighted avg       0.48      0.69      0.56       207

f1 score is-0.797342192

C:\Users\Ved\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ved\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ved\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ved\anaconda3\Lib\site-packages\sklearn\metrics\_cla

In [40]:
print(all_generation_population_and_fitness)

[[[[1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1], 11, 0.3429951690821256], [[0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1], 7, 0.3913043478260869], [[0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0], 8, 0.36714975845410625], [[0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1], 8, 0.3913043478260869], [[1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0], 8, 0.42028985507246375], [[0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0], 5, 0.3623188405797102], [[0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1], 6, 0.36714975845410625], [[0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1], 4, 0.3188405797101449], [[0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0], 6, 0.3816425120772947], [[1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1], 7, 0.36714975845410625], [[1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0], 5, 0.3623188405797102], [[0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0], 7, 0.3913043478260869], [[0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1], 9, 0.37681159420289856], [[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0], 4, 0.3381642512077294], [[1, 0, 1, 1, 0, 1, 1, 1, 